---
## 1️⃣ IMPORT LIBRARIES

In [1]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(" All libraries imported successfully!")

 All libraries imported successfully!


---
## 2️⃣ LOAD ALL 4 DATASETS

In [2]:
# Load Dataset 1: Item Master (Products)
items = pd.read_csv('annex1.csv', encoding='utf-8')
print(" Items loaded:", items.shape)
items.head()

 Items loaded: (251, 4)


,Item Code,Item Name,Category Code,Category Name
0,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables
1,102900005115199,Sichuan Red Cedar,1011010101,Flower/Leaf Vegetables
2,102900005115625,Local Xiaomao Cabbage,1011010101,Flower/Leaf Vegetables
3,102900005115748,White Caitai,1011010101,Flower/Leaf Vegetables
4,102900005115762,Amaranth,1011010101,Flower/Leaf Vegetables


In [ ]:
# Load Dataset 2: Sales Transactions
sales = pd.read_csv('annex2.csv', encoding='utf-8')
print(" Sales loaded:", sales.shape)
sales.head()

In [3]:
# Load Dataset 3: Wholesale Prices
wholesale = pd.read_csv('annex3.csv', encoding='utf-8')
print(" Wholesale loaded:", wholesale.shape)
wholesale.head()

 Wholesale loaded: (55982, 3)


,Date,Item Code,Wholesale Price (RMB/kg)
0,2020-07-01,102900005115762,3.88
1,2020-07-01,102900005115779,6.72
2,2020-07-01,102900005115786,3.19
3,2020-07-01,102900005115793,9.24
4,2020-07-01,102900005115823,7.03


In [4]:
# Load Dataset 4: Loss Rates
losses = pd.read_csv('annex4.csv', encoding='utf-8-sig')
print(" Losses loaded:", losses.shape)
losses.head()

 Losses loaded: (251, 3)


,Item Code,Item Name,Loss Rate (%)
0,102900005115168,Niushou Shengcai,4.39
1,102900005115199,Sichuan Red Cedar,10.46
2,102900005115250,Xixia Black Mushroom (1),10.80
3,102900005115625,Local Xiaomao Cabbage,0.18
4,102900005115748,White Caitai,8.78


---
## 3️⃣ UNDERSTAND EACH DATASET

In [ ]:
# Check Items dataset structure
print("=" * 70)
print("ITEMS DATASET INFO")
print("=" * 70)
items.info()
print("\nUnique Categories:", items['Category Name'].nunique())
print("\nCategory Distribution:")
print(items['Category Name'].value_counts())

In [ ]:
# Check Sales dataset structure
print("=" * 70)
print("SALES DATASET INFO")
print("=" * 70)
sales.info()
print("\nDate Range:", sales['Date'].min(), "to", sales['Date'].max())
print("\nDiscount Distribution:")
print(sales['Discount (Yes/No)'].value_counts())

In [ ]:
# Check Wholesale dataset structure
print("=" * 70)
print("WHOLESALE DATASET INFO")
print("=" * 70)
wholesale.info()

In [ ]:
# Check Losses dataset structure
print("=" * 70)
print("LOSSES DATASET INFO")
print("=" * 70)
losses.info()
print("\nLoss Rate Statistics:")
print(losses['Loss Rate (%)'].describe())

---
## 4️⃣ DATA CLEANING

In [ ]:
# Clean Sales Data
print("🧹 Cleaning Sales Data...")

# Convert date and time
sales['Date'] = pd.to_datetime(sales['Date'])
sales['DateTime'] = pd.to_datetime(sales['Date'].astype(str) + ' ' + sales['Time'])

# Extract time features
sales['Year'] = sales['Date'].dt.year
sales['Month'] = sales['Date'].dt.month
sales['MonthName'] = sales['Date'].dt.month_name()
sales['Week'] = sales['Date'].dt.isocalendar().week
sales['DayOfWeek'] = sales['Date'].dt.day_name()
sales['DayOfWeekNum'] = sales['Date'].dt.dayofweek
sales['Hour'] = sales['DateTime'].dt.hour
sales['IsWeekend'] = sales['DayOfWeekNum'].isin([5, 6]).astype(int)

# Convert to numeric
sales['Quantity Sold (kilo)'] = pd.to_numeric(sales['Quantity Sold (kilo)'], errors='coerce')
sales['Unit Selling Price (RMB/kg)'] = pd.to_numeric(sales['Unit Selling Price (RMB/kg)'], errors='coerce')

# Calculate Revenue
sales['Revenue'] = sales['Quantity Sold (kilo)'] * sales['Unit Selling Price (RMB/kg)']

# Create discount flag
sales['HasDiscount'] = (sales['Discount (Yes/No)'].str.lower() == 'yes').astype(int)

# Remove invalid records
initial_count = len(sales)
sales = sales[
    (sales['Quantity Sold (kilo)'] > 0) &
    (sales['Unit Selling Price (RMB/kg)'] > 0) &
    (sales['Revenue'] > 0)
]

print(f"Removed {initial_count - len(sales):,} invalid records")
print(f"Clean sales records: {len(sales):,}")
sales.head()

In [ ]:
# Clean Wholesale Data
print(" Cleaning Wholesale Data...")

wholesale['Date'] = pd.to_datetime(wholesale['Date'])
wholesale['Wholesale Price (RMB/kg)'] = pd.to_numeric(wholesale['Wholesale Price (RMB/kg)'], errors='coerce')
wholesale = wholesale[wholesale['Wholesale Price (RMB/kg)'] > 0]

print(f" Clean wholesale records: {len(wholesale):,}")

In [ ]:
# Clean Loss Data
print(" Cleaning Loss Data...")

# Remove % sign and spaces
losses['Loss Rate (%)'] = losses['Loss Rate (%)'].astype(str).str.replace('%', '').str.strip()
losses['Loss Rate (%)'] = pd.to_numeric(losses['Loss Rate (%)'], errors='coerce')

# Fill missing with median
median_loss = losses['Loss Rate (%)'].median()
losses['Loss Rate (%)'].fillna(median_loss, inplace=True)

print(f" Clean loss records: {len(losses):,}")
losses.head()

---
## 5️⃣ CONNECT ALL 4 DATASETS (MERGE)

In [ ]:
# STEP 1: Merge Sales + Items (get category info)
print(" Step 1: Merging Sales + Items...")
df = sales.merge(
    items[['Item Code', 'Item Name', 'Category Code', 'Category Name']],
    on='Item Code',
    how='left'
)
print(f"   Records after merge: {len(df):,}")
print(f"   Columns: {len(df.columns)}")

In [ ]:
# STEP 2: Merge with Wholesale (get cost prices)
print(" Step 2: Merging with Wholesale Prices...")
df = df.merge(
    wholesale[['Date', 'Item Code', 'Wholesale Price (RMB/kg)']],
    on=['Date', 'Item Code'],
    how='left'
)
print(f"   Records after merge: {len(df):,}")
print(f"   Columns: {len(df.columns)}")

In [ ]:
# STEP 3: Merge with Losses (get wastage rates)
print(" Step 3: Merging with Loss Rates...")
df = df.merge(
    losses[['Item Code', 'Loss Rate (%)']],
    on='Item Code',
    how='left'
)
print(f"   Records after merge: {len(df):,}")
print(f"   Columns: {len(df.columns)}")

print("\n" + "=" * 70)
print(" ALL 4 DATASETS SUCCESSFULLY CONNECTED!")
print("=" * 70)

In [ ]:
# Display the merged dataset
print("\n MASTER DATASET PREVIEW:")
df.head(10)

In [ ]:
# Check column names
print("\n All Columns in Master Dataset:")
print(df.columns.tolist())

---
## 6️⃣ CALCULATE BUSINESS METRICS

In [ ]:
# Calculate Cost, Profit, Margin, Loss
print(" Calculating Business Metrics...")

# Cost = Quantity × Wholesale Price
df['Cost'] = (df['Quantity Sold (kilo)'] * df['Wholesale Price (RMB/kg)']).fillna(0)

# Gross Profit = Revenue - Cost
df['Gross Profit'] = df['Revenue'] - df['Cost']

# Gross Margin %
df['Gross Margin %'] = ((df['Gross Profit'] / df['Revenue']) * 100).fillna(0)

# Loss Cost = Cost × Loss Rate
df['Loss Cost'] = (df['Cost'] * (df['Loss Rate (%)'] / 100)).fillna(0)

# Net Profit = Gross Profit - Loss Cost
df['Net Profit'] = df['Gross Profit'] - df['Loss Cost']

# Net Margin %
df['Net Margin %'] = ((df['Net Profit'] / df['Revenue']) * 100).fillna(0)

# Markup %
df['Markup %'] = (((df['Unit Selling Price (RMB/kg)'] - df['Wholesale Price (RMB/kg)']) /
                   df['Wholesale Price (RMB/kg)']) * 100).fillna(0)

print(" All business metrics calculated!")
print("\n New columns added:")
print("   - Cost")
print("   - Gross Profit")
print("   - Gross Margin %")
print("   - Loss Cost")
print("   - Net Profit")
print("   - Net Margin %")
print("   - Markup %")

In [ ]:
# Display sample with all metrics
df[['Item Name', 'Revenue', 'Cost', 'Gross Profit', 'Loss Cost', 'Net Profit', 'Net Margin %']].head(10)

---
## 7️⃣ DATA QUALITY CHECK

In [ ]:
# Check for missing values
print("🔍 MISSING VALUES CHECK:")
print("=" * 70)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\n🔍 DUPLICATE ROWS: {duplicates:,}")

In [ ]:
# Dataset summary
print("\n DATASET SUMMARY:")
print("=" * 70)
df.info()

---
## 8️⃣ EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
# Overall Summary Statistics
print(" OVERALL BUSINESS SUMMARY")
print("=" * 70)

total_revenue = df['Revenue'].sum()
total_cost = df['Cost'].sum()
total_gross_profit = df['Gross Profit'].sum()
total_loss_cost = df['Loss Cost'].sum()
total_net_profit = df['Net Profit'].sum()
total_transactions = len(df)
total_quantity = df['Quantity Sold (kilo)'].sum()

print(f"Total Revenue:        ¥{total_revenue:,.2f}")
print(f"Total Cost:           ¥{total_cost:,.2f}")
print(f"Total Gross Profit:   ¥{total_gross_profit:,.2f}")
print(f"Total Loss Cost:      ¥{total_loss_cost:,.2f}")
print(f"Total Net Profit:     ¥{total_net_profit:,.2f}")
print(f"\nGross Margin:        {(total_gross_profit/total_revenue)*100:.2f}%")
print(f"Net Margin:          {(total_net_profit/total_revenue)*100:.2f}%")
print(f"Loss Impact:         {(total_loss_cost/total_revenue)*100:.2f}%")
print(f"\nTotal Transactions:  {total_transactions:,}")
print(f"Total Quantity Sold: {total_quantity:,.2f} kg")
print(f"Avg Transaction:     ¥{df['Revenue'].mean():.2f}")
print(f"Avg Price:           ¥{df['Unit Selling Price (RMB/kg)'].mean():.2f}/kg")

In [ ]:
# Descriptive Statistics
print("\n📊 DESCRIPTIVE STATISTICS:")
df[['Quantity Sold (kilo)', 'Unit Selling Price (RMB/kg)', 'Revenue',
    'Gross Margin %', 'Net Margin %', 'Loss Rate (%)']].describe()

---
## 9️⃣ TOP PRODUCTS ANALYSIS

In [ ]:
# Top 20 Products by Revenue
print("🏆 TOP 20 PRODUCTS BY REVENUE")
print("=" * 70)

top_products = df.groupby('Item Name').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum',
    'Quantity Sold (kilo)': 'sum'
}).sort_values('Revenue', ascending=False).head(20)

top_products['Net Margin %'] = (top_products['Net Profit'] / top_products['Revenue'] * 100)

print(top_products)

In [ ]:
# Top 10 High Margin Products
print("\n TOP 10 HIGH MARGIN PRODUCTS")
print("=" * 70)

product_margin = df.groupby('Item Name').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum'
})
product_margin['Net Margin %'] = (product_margin['Net Profit'] / product_margin['Revenue'] * 100)

high_margin = product_margin[product_margin['Revenue'] > product_margin['Revenue'].quantile(0.5)]
print(high_margin.nlargest(10, 'Net Margin %'))

---
## 🔟 CATEGORY ANALYSIS

In [ ]:
# Category Performance
print(" CATEGORY PERFORMANCE")
print("=" * 70)

category_stats = df.groupby('Category Name').agg({
    'Revenue': 'sum',
    'Gross Profit': 'sum',
    'Net Profit': 'sum',
    'Loss Cost': 'sum',
    'Quantity Sold (kilo)': 'sum',
    'Item Code': 'nunique'
}).rename(columns={'Item Code': 'Unique Items'})

category_stats['Gross Margin %'] = (category_stats['Gross Profit'] / category_stats['Revenue'] * 100)
category_stats['Net Margin %'] = (category_stats['Net Profit'] / category_stats['Revenue'] * 100)
category_stats['Loss %'] = (category_stats['Loss Cost'] / category_stats['Revenue'] * 100)

category_stats = category_stats.sort_values('Revenue', ascending=False)
print(category_stats.head(15))

---
## 1️⃣1️⃣ TIME-BASED ANALYSIS

In [ ]:
# Daily Revenue Trend
print(" DAILY REVENUE ANALYSIS")
print("=" * 70)

daily_revenue = df.groupby('Date').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum',
    'Item Code': 'count'
}).rename(columns={'Item Code': 'Transactions'})

print(f"Average Daily Revenue: ¥{daily_revenue['Revenue'].mean():,.2f}")
print(f"Peak Revenue Day: {daily_revenue['Revenue'].idxmax()} - ¥{daily_revenue['Revenue'].max():,.2f}")
print(f"Lowest Revenue Day: {daily_revenue['Revenue'].idxmin()} - ¥{daily_revenue['Revenue'].min():,.2f}")

print("\nTop 5 Revenue Days:")
print(daily_revenue.nlargest(5, 'Revenue'))

In [ ]:
# Monthly Revenue
print("\n MONTHLY REVENUE")
print("=" * 70)

monthly_revenue = df.groupby('MonthName').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum',
    'Quantity Sold (kilo)': 'sum'
}).sort_values('Revenue', ascending=False)

print(monthly_revenue)

In [ ]:
# Day of Week Analysis
print("\n DAY OF WEEK ANALYSIS")
print("=" * 70)

dow_revenue = df.groupby('DayOfWeek').agg({
    'Revenue': 'sum',
    'Item Code': 'count'
}).rename(columns={'Item Code': 'Transactions'})

# Sort by day order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_revenue = dow_revenue.reindex(day_order)

print(dow_revenue)

In [ ]:
# Hourly Sales Pattern
print("\n HOURLY SALES PATTERN")
print("=" * 70)

hourly_revenue = df.groupby('Hour').agg({
    'Revenue': 'sum',
    'Item Code': 'count'
}).rename(columns={'Item Code': 'Transactions'})

peak_hour = hourly_revenue['Revenue'].idxmax()
print(f"Peak Sales Hour: {peak_hour}:00 - ¥{hourly_revenue.loc[peak_hour, 'Revenue']:,.2f}")
print("\nRevenue by Hour:")
print(hourly_revenue)

---
## 1️⃣2️⃣ LOSS ANALYSIS

In [ ]:
# Loss Impact Analysis
print(" LOSS & WASTAGE ANALYSIS")
print("=" * 70)

loss_by_product = df.groupby('Item Name').agg({
    'Loss Cost': 'sum',
    'Loss Rate (%)': 'mean',
    'Revenue': 'sum'
}).sort_values('Loss Cost', ascending=False)

print(f"Total Loss Cost: ¥{df['Loss Cost'].sum():,.2f}")
print(f"Average Loss Rate: {df['Loss Rate (%)'].mean():.2f}%")
print(f"Loss as % of Revenue: {(df['Loss Cost'].sum()/df['Revenue'].sum())*100:.2f}%")

print("\nTop 15 Products by Loss Cost:")
print(loss_by_product.head(15))

In [ ]:
# High Loss Rate Products
print("\n HIGH LOSS RATE PRODUCTS (>5%)")
print("=" * 70)

high_loss = loss_by_product[loss_by_product['Loss Rate (%)'] > 5].sort_values('Loss Rate (%)', ascending=False)
print(high_loss.head(20))

---
## 1️⃣3️⃣ PRICING ANALYSIS

In [ ]:
# Price Statistics
print(" PRICING ANALYSIS")
print("=" * 70)

print("Selling Price Distribution:")
print(f"  Mean: ¥{df['Unit Selling Price (RMB/kg)'].mean():.2f}/kg")
print(f"  Median: ¥{df['Unit Selling Price (RMB/kg)'].median():.2f}/kg")
print(f"  Std Dev: ¥{df['Unit Selling Price (RMB/kg)'].std():.2f}/kg")
print(f"  Min: ¥{df['Unit Selling Price (RMB/kg)'].min():.2f}/kg")
print(f"  Max: ¥{df['Unit Selling Price (RMB/kg)'].max():.2f}/kg")

print("\nMarkup Statistics:")
print(f"  Average Markup: {df['Markup %'].mean():.1f}%")
print(f"  Median Markup: {df['Markup %'].median():.1f}%")

print("\nDiscount Impact:")
discount_sales = df[df['HasDiscount'] == 1]
no_discount_sales = df[df['HasDiscount'] == 0]

print(f"  Transactions with Discount: {len(discount_sales):,} ({len(discount_sales)/len(df)*100:.1f}%)")
print(f"  Avg Revenue (with discount): ¥{discount_sales['Revenue'].mean():.2f}")
print(f"  Avg Revenue (no discount): ¥{no_discount_sales['Revenue'].mean():.2f}")

---
## 1️⃣4️⃣ CORRELATION ANALYSIS

In [ ]:
# Calculate correlation matrix
print(" CORRELATION ANALYSIS")
print("=" * 70)

corr_cols = ['Quantity Sold (kilo)', 'Unit Selling Price (RMB/kg)', 'Revenue',
             'Gross Margin %', 'Net Margin %', 'Loss Rate (%)', 'Markup %']

correlation_matrix = df[corr_cols].corr()
print(correlation_matrix)

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            fmt='.2f', linewidths=1, square=True)
plt.title('Correlation Matrix - Key Metrics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 1️⃣5️⃣ VISUALIZATIONS

In [ ]:
# 1. Daily Revenue Trend
daily_data = df.groupby('Date')['Revenue'].sum().reset_index()

fig = px.line(daily_data, x='Date', y='Revenue',
              title='Daily Revenue Trend',
              labels={'Revenue': 'Revenue (¥)', 'Date': 'Date'})
fig.update_layout(hovermode='x unified', template='plotly_white', height=500)
fig.show()

In [ ]:
# 2. Top 15 Products Revenue Bar Chart
top15 = df.groupby('Item Name')['Revenue'].sum().nlargest(15).reset_index()

fig = px.bar(top15, x='Revenue', y='Item Name', orientation='h',
             title='Top 15 Products by Revenue',
             labels={'Revenue': 'Revenue (¥)', 'Item Name': 'Product'})
fig.update_layout(yaxis={'categoryorder':'total ascending'}, template='plotly_white', height=600)
fig.show()

In [ ]:
# 3. Category Performance
cat_data = df.groupby('Category Name').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum'
}).nlargest(15, 'Revenue').reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(name='Revenue', x=cat_data['Category Name'], y=cat_data['Revenue'], marker_color='skyblue'))
fig.add_trace(go.Bar(name='Net Profit', x=cat_data['Category Name'], y=cat_data['Net Profit'], marker_color='green'))

fig.update_layout(title='Top 15 Categories: Revenue vs Net Profit',
                  xaxis_title='Category', yaxis_title='Amount (¥)',
                  barmode='group', template='plotly_white', height=600,
                  xaxis_tickangle=-45)
fig.show()

In [ ]:
# 4. Hourly Sales Heatmap
hourly_dow = df.groupby(['DayOfWeekNum', 'Hour'])['Revenue'].sum().reset_index()
pivot_data = hourly_dow.pivot(index='DayOfWeekNum', columns='Hour', values='Revenue')

day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
pivot_data.index = day_names

fig = px.imshow(pivot_data,
                labels=dict(x="Hour of Day", y="Day of Week", color="Revenue (¥)"),
                title="Sales Heatmap: Day of Week vs Hour",
                aspect="auto", color_continuous_scale='YlOrRd')
fig.update_layout(template='plotly_white', height=500)
fig.show()

In [ ]:
# 5. Price Distribution
fig = px.histogram(df, x='Unit Selling Price (RMB/kg)', nbins=50,
                   title='Price Distribution',
                   labels={'Unit Selling Price (RMB/kg)': 'Price (¥/kg)'})
fig.update_layout(template='plotly_white', height=500, showlegend=False)
fig.show()

In [ ]:
# 6. Loss Analysis Chart
loss_top20 = df.groupby('Item Name')['Loss Cost'].sum().nlargest(20).reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(x=loss_top20['Item Name'], y=loss_top20['Loss Cost'],
                     marker_color='crimson',
                     hovertemplate='<b>%{x}</b><br>Loss: ¥%{y:,.2f}<extra></extra>'))

fig.update_layout(title='Top 20 Items by Loss Cost',
                  xaxis_title='Product', yaxis_title='Loss Cost (¥)',
                  template='plotly_white', height=600, xaxis_tickangle=-45)
fig.show()

In [ ]:
# 7. Revenue vs Margin Scatter Plot
product_stats = df.groupby('Item Name').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum',
    'Quantity Sold (kilo)': 'sum',
    'Category Name': 'first'
}).reset_index()

product_stats['Net Margin %'] = (product_stats['Net Profit'] / product_stats['Revenue'] * 100)

# Filter for better visualization
plot_data = product_stats[product_stats['Revenue'] > product_stats['Revenue'].quantile(0.25)]

fig = px.scatter(plot_data, x='Revenue', y='Net Margin %',
                 size='Quantity Sold (kilo)', color='Category Name',
                 hover_name='Item Name',
                 title='Revenue vs Margin Analysis (by Category)',
                 labels={'Revenue': 'Revenue (¥)', 'Net Margin %': 'Net Margin (%)'})
fig.update_layout(template='plotly_white', height=600)
fig.show()

In [ ]:
# 8. Monthly Revenue Comparison
monthly_data = df.groupby('MonthName').agg({
    'Revenue': 'sum',
    'Net Profit': 'sum'
}).reset_index()

month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_data['MonthName'] = pd.Categorical(monthly_data['MonthName'], categories=month_order, ordered=True)
monthly_data = monthly_data.sort_values('MonthName')

fig = go.Figure()
fig.add_trace(go.Bar(name='Revenue', x=monthly_data['MonthName'], y=monthly_data['Revenue'], marker_color='lightblue'))
fig.add_trace(go.Bar(name='Net Profit', x=monthly_data['MonthName'], y=monthly_data['Net Profit'], marker_color='green'))

fig.update_layout(title='Monthly Revenue & Profit Comparison',
                  xaxis_title='Month', yaxis_title='Amount (¥)',
                  barmode='group', template='plotly_white', height=500)
fig.show()

---
## 1️⃣6️⃣ KEY INSIGHTS & RECOMMENDATIONS

In [ ]:
# Generate Key Insights
print(" KEY BUSINESS INSIGHTS")
print("=" * 70)

# 1. Revenue concentration
top5_items = df.groupby('Item Name')['Revenue'].sum().nlargest(5)
top5_pct = (top5_items.sum() / df['Revenue'].sum()) * 100
print(f"\n1. REVENUE CONCENTRATION")
print(f"   Top 5 items account for {top5_pct:.1f}% of total revenue")
print(f"   Focus on these high-performers for growth")

# 2. Profitability
avg_margin = (df['Net Profit'].sum() / df['Revenue'].sum()) * 100
print(f"\n2. PROFITABILITY")
print(f"   Overall net margin: {avg_margin:.2f}%")
if avg_margin < 20:
    print(f"   Margin below 20% - consider price optimization")
else:
    print(f"    Healthy margin above 20%")

# 3. Loss impact
loss_pct = (df['Loss Cost'].sum() / df['Revenue'].sum()) * 100
print(f"\n3. LOSS IMPACT")
print(f"   Wastage costs: ¥{df['Loss Cost'].sum():,.2f} ({loss_pct:.2f}% of revenue)")
if loss_pct > 5:
    print(f"    High loss rate - implement better inventory management")

# 4. Time patterns
dow_sales = df.groupby('DayOfWeek')['Revenue'].sum()
best_day = dow_sales.idxmax()
worst_day = dow_sales.idxmin()
print(f"\n4. WEEKLY PATTERN")
print(f"   Best day: {best_day}")
print(f"   Worst day: {worst_day}")
print(f"    Recommendation: Run promotions on {worst_day} to boost sales")

# 5. Peak hours
hourly = df.groupby('Hour')['Revenue'].sum()
peak_hour = hourly.idxmax()
print(f"\n5. PEAK HOURS")
print(f"   Peak sales hour: {peak_hour}:00")
print(f"    Recommendation: Ensure adequate staffing during peak hours")

# 6. High loss items
high_loss_items = df.groupby('Item Name').agg({
    'Loss Cost': 'sum',
    'Loss Rate (%)': 'mean'
}).nlargest(5, 'Loss Cost')
print(f"\n6. HIGH LOSS ITEMS (Top 5)")
for item, row in high_loss_items.iterrows():
    print(f"   {item}: ¥{row['Loss Cost']:,.2f} loss ({row['Loss Rate (%)']:.1f}%)")
print(f"   💡 Recommendation: Review storage/handling for these items")

print("\n" + "=" * 70)
print(" ANALYSIS COMPLETE!")
print("=" * 70)

---
## 1️⃣7️⃣ SAVE RESULTS

In [ ]:
# Save the master dataset
df.to_csv('master_dataset.csv', index=False, encoding='utf-8')
print(" Master dataset saved: master_dataset.csv")

# Save key summary tables
category_stats.to_csv('category_performance.csv')
print("Category performance saved: category_performance.csv")

top_products.to_csv('top_products.csv')
print(" Top products saved: top_products.csv")

loss_by_product.to_csv('loss_analysis.csv')
print(" Loss analysis saved: loss_analysis.csv")

print("\n🎉 All results saved successfully!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')